In [1]:
# https://pytorch.org/ 依官網所示
!pip3 install torch torchvision torchaudio

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 2.3/2.3 MB 4.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from transformers import AutoTokenizer, BertTokenizerFast,AutoModelForCausalLM
#----->沒使用pipeline,自行設定套件

1. gpt2-small: The smallest version, with 117 million parameters.
2. gpt2-medium: The medium-sized version, with 345 million parameters.
3. gpt2-large: The large version, with 774 million parameters.
4. gpt2-xl: The largest version, with 1.5 billion parameters.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
#產生句子
input_txt = "I have a pen, I have an "
# tokenizer()會返回單詞編號:input_ids/類型:token_type_ids/是否mask:attention_mask
#'tf'：傳回 TensorFlow tf.constant 物件。'pt'：傳回 PyTorch torch.Tensor 物件。'np'：傳回 Numpy np.ndarray 物件。
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"]#.to(device)
iterations = []
n_steps = 10
choices_per_step = 3

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids)
        # 選最後一個 token 然後過 softmax 後選出機率最大
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

print(iterations[-1])

2023-12-21 15:25:15.515694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


{'Input': 'I have a pen, I have an iphone, I have a laptop, I'}


In [4]:
max_length = 64

input_txt = """I have a pen, I have an iphone, I have a laptop. Thus,"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_length=max_length)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I have a pen, I have an iphone, I have a laptop. Thus, I have a lot of things that I can do. I can do a lot of things. I can do a lot of things. I can do a lot of things. I can do a lot of things. I can


### 中文

In [5]:

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese')
#.to(device)

In [6]:
max_length=256
input_txt = """
隨著貸款日益枯竭，Alistair Darling 被迫考慮對銀行進行第二次救助。 \
財政大臣將在幾週內決定是否向經濟中再注入數十億美元，因為有證據表明\
去年 370 億的部分國有化未能保持信貸流動，
"""

In [7]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"]#.to(device)
output = model.generate(input_ids, max_length=max_length, num_beams=1,  do_sample=True, top_k=50)
print(tokenizer.decode(output[0]))
#print(tokenizer.decode(output[0], skip_special_tokens=True))# 去掉 Special Token ID

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 隨 著 貸 款 日 益 枯 竭 ， [UNK] [UNK] 被 迫 考 慮 對 銀 行 進 行 第 二 次 救 助 。 財 政 大 臣 將 在 幾 週 內 決 定 是 否 向 經 濟 中 再 注 入 數 十 億 美 元 ， 因 為 有 證 據 表 明 去 年 370 億 的 部 分 國 有 化 未 能 保 持 信 貸 流 動 ， [SEP] 得 它 自 身 對 房 地 產 的 依 賴 程 度 增 加 。 [UNK] [UNK] [UNK] （ ） 是 香 港 一 個 以 [UNK] [UNK] 為 發 售 的 免 費 電 訊 軟 體 平 臺 ， 由 香 港 無 綫 電 視 聯 播 有 限 公 司 製 作 ， 由 陳 奕 迅 、 譚 嘉 銘 、 陳 浩 然 、 周 潤 發 、 陳 麗 芬 及 [UNK] [UNK] 發 起 。 目 前 為 一 個 平 臺 ， 於 2017 年 6 月 31 日 起 每 平 方 英 文 電 腦 下 載 （ [UNK] + 1 ） ， 但 由 於 當 時 是 有 線 電 視 的 開 放 ， 此 舉 造 成 消 費 者 的 困 擾 ， 令 不 少 網 民 不 能 安 心 到 此 ， 為 了 滿 足 電 訊 廣 播 服 務 商 的 需 求 ， 香 港 電 話 臺 每 天 均 進


In [23]:
doc = tokenizer.decode(output[0])
doc

'[CLS] 隨 著 貸 款 日 益 枯 竭 ， [UNK] [UNK] 被 迫 考 慮 對 銀 行 進 行 第 二 次 救 助 。 財 政 大 臣 將 在 幾 週 內 決 定 是 否 向 經 濟 中 再 注 入 數 十 億 美 元 ， 因 為 有 證 據 表 明 去 年 370 億 的 部 分 國 有 化 未 能 保 持 信 貸 流 動 ， [SEP] 得 它 自 身 對 房 地 產 的 依 賴 程 度 增 加 。 [UNK] [UNK] [UNK] （ ） 是 香 港 一 個 以 [UNK] [UNK] 為 發 售 的 免 費 電 訊 軟 體 平 臺 ， 由 香 港 無 綫 電 視 聯 播 有 限 公 司 製 作 ， 由 陳 奕 迅 、 譚 嘉 銘 、 陳 浩 然 、 周 潤 發 、 陳 麗 芬 及 [UNK] [UNK] 發 起 。 目 前 為 一 個 平 臺 ， 於 2017 年 6 月 31 日 起 每 平 方 英 文 電 腦 下 載 （ [UNK] + 1 ） ， 但 由 於 當 時 是 有 線 電 視 的 開 放 ， 此 舉 造 成 消 費 者 的 困 擾 ， 令 不 少 網 民 不 能 安 心 到 此 ， 為 了 滿 足 電 訊 廣 播 服 務 商 的 需 求 ， 香 港 電 話 臺 每 天 均 進'